In [1]:
from thermo import ChemicalConstantsPackage
from thermo.unifac import UNIFAC, DOUFSG, DOUFIP2016
from chemicals import search_chemical
import numpy as np
import pandas as pd
from IPython.display import clear_output

In [2]:
with pd.ExcelFile('Sorted Data.xlsx') as f:
    df = pd.read_excel(f, 'Data')

In [4]:
y = []
IUPAC_mix = []
all_chem_mix = []
for i in range(len(df)):
    print(f'Datapoint {i+1} out of {len(df)}')
    c1 = df['Component 1'][i]
    c2 = df['Component 2'][i]
    IUPAC_mix += [[c1, c2]]
    constants,_ = ChemicalConstantsPackage.from_IDs(IUPAC_mix[i])
    all_chem_mix += [constants.UNIFAC_Dortmund_groups]
    x = df['Composition component 1 [mol/mol]'][i]
    xs = [x, 1-x]
    T = df['Temperature [K]'][i]
    GE = UNIFAC.from_subgroups(T=T, xs=xs, chemgroups=all_chem_mix[i], version=1, interaction_data=DOUFIP2016, subgroups=DOUFSG)
    y += [GE.HE()]
    clear_output(wait=True)

Datapoint 28214 out of 28214


In [8]:
data = {'Component 1': np.array(IUPAC_mix)[:,0],
        'Component 2': np.array(IUPAC_mix)[:,1],
        'UNIFAC_DMD [J/mol]': y}
data = pd.DataFrame(data)
data.to_excel('Thermo_UNIFAC_DMD.xlsx', index=False, header=True, sheet_name='UNIFAC_DMD')

In [7]:
np.array(IUPAC_mix)

array([['Methanoic acid', 'Methanol'],
       ['Methanoic acid', 'Methanol'],
       ['Methanoic acid', 'Methanol'],
       ...,
       ['1-Decanol', 'tert-Butanol'],
       ['1-Decanol', 'tert-Butanol'],
       ['1-Decanol', 'tert-Butanol']], dtype='<U17')